In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np

# load the MNIST data EXACTLY as required, but using the stable parser
X_mnist, y_mnist = fetch_openml(
    data_id=554,
    return_X_y=True,
    as_frame=False,
    parser="auto"     # <-- fixes the network retry problem
)

X = X_mnist.astype(np.float32) / 255.0
y = y_mnist.astype(int)

print("X shape:", X.shape)
print("y shape:", y.shape)
